https://www.data.go.kr/iim/api/selectAPIAcountView.do


https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057988

### 요청변수(Request Parameter)

| 항목명(국문) | 항목명(영문)   | 항목크기 | 항목구분 | 샘플데이터               | 항목설명                 |
|---------|-----------|------|------|---------------------|----------------------|
| 고유번호    | pnu       | 19   | 필수   | 1111010100100040007 | 고유번호(8자리 이상)         |
| 기준년도    | stdrYear  | 4    | 옵션   | 2012                | 기준년도(YYYY: 4자리)      |
| 응답결과 형식 | format    | 5    | 옵션   | xml                 | 응답결과 형식(xml 또는 json) |
| 검색건수    | numOfRows | 4    | 옵션   | 10                  | 검색건수                 |
| 페이지 번호  | pageNo    | 4    | 옵션   | 1                   | 페이지 번호               |


### 출력결과(Response Element)

| 항목명(국문)  | 항목명(영문)        | 항목크기 | 항목구분 | 샘플데이터               | 항목설명     |
|----------|----------------|------|------|---------------------|----------|
| 고유번호     | pnu            | 19   | 필수   | 1111010100100040007 | 고유번호     |
| 법정동코드    | ldCode         | 10   | 옵션   | 1111010100          | 법정동코드    |
| 법정동명     | ldCodeNm       | 300  | 옵션   | 서울특별시 종로구 청운동       | 법정동명     |
| 특수지구분코드  | regstrSeCode   | 1    | 옵션   | 1                   | 특수지구분코드  |
| 특수지구분명   | regstrSeCodeNm | 300  | 옵션   | 일반                  | 특수지구분명   |
| 지번       | mnnmSlno       | 9    | 옵션   | 1-1                 | 지번       |
| 기준년도     | stdrYear       | 4    | 필수   | 2012                | 기준년도     |
| 기준월      | stdrMt         | 2    | 필수   | 09                  | 기준월      |
| 공동주택코드   | aphusCode      | 10   | 옵션   | 1                   | 공동주택코드   |
| 공동주택구분코드 | aphusSeCode    | 1    | 옵션   | 5                   | 공동주택구분코드 |
| 공동주택구분명  | aphusSeCodeNm  | 300  | 옵션   | 다세대                 | 공동주택구분명  |
| 특수지명     | spclLandNm     | 150  | 옵션   | 상암택지개발사업지구 3-7블럭    | 특수지명     |
| 공동주택명    | aphusNm        | 300  | 옵션   | 상암월드컵7단지            | 공동주택명    |
| 전용면적(㎡)  | prvuseAr       | 20   | 옵션   | 84.45               | 전용면적(㎡)  |
| 공시가격(원)  | pblntfPc       | 13   | 옵션   | 480000000           | 공시가격(원)  |
| 데이터기준일자  | lastUpdtDt     | 10   | 옵션   | 2016-09-26          | 데이터기준일자  |


In [18]:
filename = 'ServiceKey.txt'
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)
api_key = get_file_contents(filename)
#print("Our API key is: %s" % (api_key))

In [19]:
import requests


url = 'http://apis.data.go.kr/1611000/nsdi/ApartHousingPriceService/attr/getApartHousingPriceAttr'


params ={'serviceKey' : api_key,
         'pnu' : '1111010100100040007', 
         'stdrYear' : '2012', 
         'format' : 'xml', 
         'numOfRows' : '10', 
         'pageNo' : '1' }

response = requests.get(url, params=params).text.encode('utf-8')
#response
#print(response.content)


In [31]:
import bs4 as bs
import urllib.request
import pandas as pd 

soup = bs.BeautifulSoup(response,'xml')


soup.find_all()

In [21]:
import xmltodict
 
data = xmltodict.parse(response)

data

In [22]:
rows = soup.find_all('field')
rows

[<field><pnu>1111010100100040007</pnu><ldCode>1111010100</ldCode><ldCodeNm>서울특별시 종로구 청운동</ldCodeNm><regstrSeCode>1</regstrSeCode><regstrSeCodeNm>일반</regstrSeCodeNm><mnnmSlno>4-7</mnnmSlno><stdrYear>2012</stdrYear><stdrMt>01</stdrMt><aphusCode>1</aphusCode><aphusSeCode>5</aphusSeCode><aphusSeCodeNm>다세대</aphusSeCodeNm><aphusNm>신구파인힐</aphusNm><floorNm>1</floorNm><hoNm>101</hoNm><prvuseAr>211.13</prvuseAr><pblntfPc>700000000</pblntfPc><lastUpdtDt>2016-11-09</lastUpdtDt></field>,
 <field><pnu>1111010100100040007</pnu><ldCode>1111010100</ldCode><ldCodeNm>서울특별시 종로구 청운동</ldCodeNm><regstrSeCode>1</regstrSeCode><regstrSeCodeNm>일반</regstrSeCodeNm><mnnmSlno>4-7</mnnmSlno><stdrYear>2012</stdrYear><stdrMt>01</stdrMt><aphusCode>1</aphusCode><aphusSeCode>5</aphusSeCode><aphusSeCodeNm>다세대</aphusSeCodeNm><aphusNm>신구파인힐</aphusNm><floorNm>1</floorNm><hoNm>102</hoNm><prvuseAr>209.21</prvuseAr><pblntfPc>691000000</pblntfPc><lastUpdtDt>2016-11-09</lastUpdtDt></field>,
 <field><pnu>1111010100100040007</pnu><l

In [23]:
columns = rows[0].find_all()
columns

[<pnu>1111010100100040007</pnu>,
 <ldCode>1111010100</ldCode>,
 <ldCodeNm>서울특별시 종로구 청운동</ldCodeNm>,
 <regstrSeCode>1</regstrSeCode>,
 <regstrSeCodeNm>일반</regstrSeCodeNm>,
 <mnnmSlno>4-7</mnnmSlno>,
 <stdrYear>2012</stdrYear>,
 <stdrMt>01</stdrMt>,
 <aphusCode>1</aphusCode>,
 <aphusSeCode>5</aphusSeCode>,
 <aphusSeCodeNm>다세대</aphusSeCodeNm>,
 <aphusNm>신구파인힐</aphusNm>,
 <floorNm>1</floorNm>,
 <hoNm>101</hoNm>,
 <prvuseAr>211.13</prvuseAr>,
 <pblntfPc>700000000</pblntfPc>,
 <lastUpdtDt>2016-11-09</lastUpdtDt>]

In [24]:
columns[0].name, columns[0].text

('pnu', '1111010100100040007')

In [25]:
rowList = []
nameList = []
columnList = []

columnsLen = len(columns)
for j in range(0, columnsLen):
    eachColumn = columns[j].text
    columnList.append(eachColumn)
rowList.append(columnList)

In [26]:
rowList

[['1111010100100040007',
  '1111010100',
  '서울특별시 종로구 청운동',
  '1',
  '일반',
  '4-7',
  '2012',
  '01',
  '1',
  '5',
  '다세대',
  '신구파인힐',
  '1',
  '101',
  '211.13',
  '700000000',
  '2016-11-09']]

In [32]:

rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
        if i == 0:
            nameList.append(columns[j].name)
        # 컬럼값은 모든 행의 값을 저장해야한다.    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
    
result = pd.DataFrame(rowList, columns=nameList)
result.head()

,pnu,ldCode,ldCodeNm,regstrSeCode,regstrSeCodeNm,mnnmSlno,stdrYear,stdrMt,aphusCode,aphusSeCode,aphusSeCodeNm,aphusNm,floorNm,hoNm,prvuseAr,pblntfPc,lastUpdtDt
0,1111010100100040007,1111010100,서울특별시 종로구 청운동,1,일반,4-7,2012,01,1,5,다세대,신구파인힐,1,101,211.13,700000000,2016-11-09
1,1111010100100040007,1111010100,서울특별시 종로구 청운동,1,일반,4-7,2012,01,1,5,다세대,신구파인힐,1,102,209.21,691000000,2016-11-09
2,1111010100100040007,1111010100,서울특별시 종로구 청운동,1,일반,4-7,2012,01,1,5,다세대,신구파인힐,2,201,207.89,691000000,2016-11-09
3,1111010100100040007,1111010100,서울특별시 종로구 청운동,1,일반,4-7,2012,01,1,5,다세대,신구파인힐,2,202,215.09,700000000,2016-11-09
4,1111010100100040007,1111010100,서울특별시 종로구 청운동,1,일반,4-7,2012,01,1,5,다세대,신구파인힐,3,301,219.33,729000000,2016-11-09


In [33]:
result.describe()

,pnu,ldCode,ldCodeNm,regstrSeCode,regstrSeCodeNm,mnnmSlno,stdrYear,stdrMt,aphusCode,aphusSeCode,aphusSeCodeNm,aphusNm,floorNm,hoNm,prvuseAr,pblntfPc,lastUpdtDt
count,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
unique,1,1,1,1,1,1,1,1,1,1,1,1,4,8,8,5,1
top,1111010100100040007,1111010100,서울특별시 종로구 청운동,1,일반,4-7,2012,01,1,5,다세대,신구파인힐,2,201,211.13,700000000,2016-11-09
freq,8,8,8,8,8,8,8,8,8,8,8,8,2,1,1,3,8


In [34]:
result.shape

(8, 17)

In [35]:
result.columns

Index(['pnu', 'ldCode', 'ldCodeNm', 'regstrSeCode', 'regstrSeCodeNm',
       'mnnmSlno', 'stdrYear', 'stdrMt', 'aphusCode', 'aphusSeCode',
       'aphusSeCodeNm', 'aphusNm', 'floorNm', 'hoNm', 'prvuseAr', 'pblntfPc',
       'lastUpdtDt'],
      dtype='object')

import pandas as pd
pd.DataFrame.from_dict(data)

import xmltodict, json

json.dumps(data) # '{"e": {"a": ["text", "text"]}}'

from pandas import json_normalize
info = json.loads(json.dumps(data))

df = json_normalize(info['wfs:FeatureCollection']) #Results contain the required data
print(df)